In [1]:
import pandas as pd

data = pd.read_parquet('../cache/btc.parquet')
data.head()

,Timestamp,Open,High,Low,Close,Volume
0,1.325412e+09,4.58,4.58,4.58,4.58,0.0
1,1.325412e+09,4.58,4.58,4.58,4.58,0.0
2,1.325412e+09,4.58,4.58,4.58,4.58,0.0
3,1.325412e+09,4.58,4.58,4.58,4.58,0.0
4,1.325412e+09,4.58,4.58,4.58,4.58,0.0


In [2]:
data.columns

Index(['Timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object')

In [3]:
# drop NaN values
data.dropna(inplace=True)

In [4]:
from datetime import datetime

data['timestamp'] = data['Timestamp'].astype('int64') // 10**9

# get data from ranges 2019-01-01 to 2023-12-31
timeStamp1 = pd.Timestamp('2019-01-01').value // 10**9
timeStamp2 = pd.Timestamp('2024-01-01').value // 10**9

data_ranged = data[(data['Timestamp'] >= timeStamp1) & (data['Timestamp'] <= timeStamp2)]
data_ranged.head()

,Timestamp,Open,High,Low,Close,Volume,timestamp
3681059,1.546301e+09,3750.62,3752.01,3750.62,3752.01,1.243647,1
3681060,1.546301e+09,3752.01,3752.01,3752.01,3752.01,0.000000,1
3681061,1.546301e+09,3752.01,3752.01,3752.01,3752.01,0.000000,1
3681062,1.546301e+09,3752.01,3752.01,3752.01,3752.01,0.000000,1
3681063,1.546301e+09,3750.31,3750.31,3750.31,3750.31,0.359107,1
